In [ ]:
import json
import pandas as pd
from pymongo import MongoClient

In [ ]:
client = MongoClient('localhost', 27017)
db = client.steamspy

In [ ]:
data = pd.DataFrame(list(db.clean.find()))

In [ ]:
data.drop(columns = "_id", inplace = True) #drop _id column

In [ ]:
gamefeatures= pd.read_csv("gamefeatures.csv")

In [ ]:
gamefeatures = gamefeatures.rename(columns = {'QueryID':'id'})

In [ ]:
gamefeatures = gamefeatures.drop(columns = ['ResponseID', 'QueryName', 'ResponseName', 'DemoCount', 'DeveloperCount', 'DLCCount',
                            'MovieCount', 'PackageCount', 'PublisherCount', 'ScreenshotCount',
                            'SteamSpyOwners', 'SteamSpyOwnersVariance', 'SteamSpyPlayersEstimate',
                            'SteamSpyPlayersVariance', 'AchievementHighlightedCount', 'IsFree', 'FreeVerAvail',
                            'PurchaseAvail', 'PCReqsHaveMin', 'PCReqsHaveRec', 'LinuxReqsHaveMin', 
                            'LinuxReqsHaveRec', 'MacReqsHaveMin', 'MacReqsHaveRec', 'CategorySinglePlayer',
                            'CategoryMultiplayer', 'CategoryCoop', 'CategoryMMO', 'CategoryInAppPurchase',
                            'CategoryIncludeSrcSDK', 'CategoryIncludeLevelEditor', 'CategoryVRSupport', 
                            'GenreIsNonGame', 'GenreIsIndie', 'GenreIsAction', 'GenreIsAdventure', 'GenreIsCasual',
                            'GenreIsStrategy', 'GenreIsRPG', 'GenreIsSimulation', 'GenreIsEarlyAccess', 
                            'GenreIsFreeToPlay', 'GenreIsSports', 'GenreIsRacing', 'GenreIsMassivelyMultiplayer', 
                            'PriceCurrency', 'PriceInitial', 'PriceFinal', 'SupportEmail', 'SupportURL', 
                            'AboutText', 'Background', 'ShortDescrip', 'DetailedDescrip', 'DRMNotice', 
                            'ExtUserAcctNotice', 'HeaderImage', 'LegalNotice', 'Reviews', 'SupportedLanguages',
                            'Website', 'PCMinReqsText', 'PCRecReqsText', 'LinuxMinReqsText', 'LinuxRecReqsText',
                            'MacMinReqsText', 'MacRecReqsText']) #dropping useless columns

In [ ]:
gamefeatures = gamefeatures.rename(columns = {'id':'appid'}) #match datas id column

In [ ]:
df = pd.merge(gamefeatures, data, on=['appid'], how='left')

In [ ]:
merged = db.merged

In [ ]:
records = json.loads(df.T.to_json()).values() #store the merged df in mongo
merged.insert(records)

In [286]:
df.head()

,appid,ReleaseDate,RequiredAge,Metacritic,RecommendationCount,AchievementCount,ControllerSupport,SubscriptionAvail,PlatformWindows,PlatformLinux,...,median_2weeks,median_forever,name,negative,positive,price,publisher,score_rank,ulOwners,userscore
0,10,Nov 1 2000,0,88,68991,0,False,False,True,True,...,36.0,166.0,Counter-Strike,3754.0,144330.0,999,Valve,,50000000.0,0.0
1,20,Apr 1 1999,0,0,2439,0,False,False,True,True,...,143.0,36.0,Team Fortress Classic,711.0,3804.0,499,Valve,,5000000.0,0.0
2,30,May 1 2003,0,79,2319,0,False,False,True,True,...,143.0,28.0,Day of Defeat,468.0,3909.0,499,Valve,,10000000.0,0.0
3,40,Jun 1 2001,0,0,888,0,False,False,True,True,...,143.0,15.0,Deathmatch Classic,317.0,1469.0,499,Valve,,10000000.0,0.0
4,50,Nov 1 1999,0,0,2934,0,False,False,True,True,...,143.0,121.0,Half-Life: Opposing Force,359.0,6529.0,499,Valve,,10000000.0,0.0


In [ ]:
free = data[data['Genres'].apply(lambda x: 'Free' in x)] 
paid = data[data['Genres'].apply(lambda x: 'Free' not in x)] 

In [ ]:
freeOwnersL = free['llOwners'].sum()
paidOwnersL = paid['llOwners'].sum()
freeOwnersU = free['ulOwners'].sum()
paidOwnersU = paid['ulOwners'].sum()

In [ ]:
dfFree = pd.DataFrame({'Type':['Free', 'Paid'], 'Lower Limit Owners':[freeOwnersL, paidOwnersL], 'Upper Limit Owners':
                      [freeOwnersU, paidOwnersU]})

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ax = dfFree.plot.bar(x = 'Type',rot=0) # number of free and paid games owned by people

In [ ]:
#takes in a list of dataframes and outputs the values of the amount of owners and counts
def getOwnersCount(listOfDfs):
    lowerOwners = []
    upperOwners = []
    count = []
    for i in listOfDfs:
        lowerOwners.append(i['llOwners'].sum())
        upperOwners.append(i['ulOwners'].sum())
        count.append(i.shape[0])
    return lowerOwners, upperOwners, count

In [ ]:
#takes in a list of dataframes and names for the dataframes and outputs a new dataframe with owners and count 
def createOwnersDF(listOfDfs, names):
    lO, uO, count = getOwnersCount(listOfDfs)
    df = pd.DataFrame({'Type':names,'Lower Limit Owners':lO, 'Upper Limit Owners': uO, 'Count':count})
    df['llAvg'] = df['Lower Limit Owners']/df['Count']
    df['ulAvg'] = df['Upper Limit Owners']/df['Count']
    return df

In [265]:
def plotOwners(listOfDfs, names, rotation=0):
    df = createOwnersDF(listOfDfs, names)
    ax = df.plot.bar(x = 'Type',y = ['Lower Limit Owners', 'Upper Limit Owners'],rot=rotation, 
                     figsize = (20,10), title="Number of Owners") 
    ax = df.plot.bar(x = 'Type',y = 'Count',rot=rotation, figsize = (20,10), title = "Number of Games")
    ax = df.plot.bar(x = 'Type',y = ['llAvg','ulAvg'],rot=rotation, figsize=(20,10), title = "Average owners per game")

In [274]:
#takes in a list of dataframes and outputs the values of avg and median new users and counts
def getAvgMedCount(listOfDfs):
    avg = []
    median = []
    count = []
    for i in listOfDfs:
        avg.append(i['average_forever'].sum())
        median.append(i['median_forever'].sum())
        count.append(i.shape[0])
    return avg, median, count

In [278]:
#takes in a list of dataframes and names for the dataframes and outputs a new dataframe with new users and count 
def createAvgMedDF(listOfDfs, names):
    avg, med, count = getAvgMedCount(listOfDfs)
    df = pd.DataFrame({'Type':names,'Avg New Users':avg, 'Median New Users': med, 'Count':count})
    df['avgAvg'] = df['Avg New Users']/df['Count']
    df['medAvg'] = df['Median New Users']/df['Count']
    return df

In [228]:
#plotting the dataframe
def plotNewOwners(listOfDfs, names, rotation=0):
    df = createAvgMedDF(listOfDfs, names)
    ax = df.plot.bar(x = 'Type',y = ['Avg New Users', 'Median New Users'],rot=rotation, 
                     figsize = (20,10), title="Number of New Owners") 
    ax = df.plot.bar(x = 'Type',y = ['avgAvg','medAvg'],rot=rotation, figsize=(20,10), 
                     title = "Average new owners per game")

In [266]:
action = data[data['Genres'].apply(lambda x: 'Action' in x)]
earlyAccess = data[data['Genres'].apply(lambda x: 'Early Access' in x)]
strategy = data[data['Genres'].apply(lambda x: 'Strategy' in x)]
rpg = data[data['Genres'].apply(lambda x: 'RPG' in x)]
indie = data[data['Genres'].apply(lambda x: 'Indie' in x)]
adventure = data[data['Genres'].apply(lambda x: 'Adventure' in x)]
sports = data[data['Genres'].apply(lambda x: 'Sports' in x)]
simulation = data[data['Genres'].apply(lambda x: 'Simulation' in x)]
exEarlyAccess = data[data['Genres'].apply(lambda x: 'Ex Early Access' in x)]
mmo = data[data['Genres'].apply(lambda x: 'MMO' in x)]
theGenres = [action, earlyAccess, strategy, rpg, indie, adventure, sports, simulation, exEarlyAccess, mmo]
genreNames = ['Action', 'Early Access', 'Strategy', 'RPG', 'Indie', 'Adventure', 'Sports',
              'Simulation', 'Ex Early Access', 'MMO']

In [ ]:
#making price column ints and making price of free games 0
data['price'] = data['price'].fillna(0)
data['price'] = data['price'].astype(int)

In [ ]:
data['negative'] = data['negative'].astype(int)
data['positive'] = data['positive'].astype(int)

In [ ]:
data['Like/Dislike Ratio'] = data['positive']/data['negative']

In [ ]:
free = data[data.price == 0]
paid10 = data[(data.price < 1001) & (data.price > 0)] #games between 0.01-10 dollars
paid20 = data[(data.price > 1000) & (data.price < 2001)]
paid30 = data[(data.price > 2000) & (data.price < 3001)]
paid40 = data[(data.price > 3000) & (data.price < 4001)]
paid50 = data[(data.price > 4000) & (data.price < 5001)]
paid60 = data[(data.price > 5000) & (data.price < 6001)]
paidOver60 = data[data.price > 6000]

In [ ]:
priceOfGame = [free, paid10, paid20, paid30, paid40, paid50, paid60, paidOver60]
priceNames = ['Free', 'Paid 10', 'Paid 20', 'Paid 30', 'Paid 40', 'Paid 50', 'Paid 60', 'Paid Over 60']

In [ ]:
dfPaid = createOwnersDF(priceOfGame, priceNames)

In [ ]:
#owners by price point without truncation
plotOwners(dfPaid)

In [ ]:
ordereddata = data.sort_values(by=['llOwners','ulOwners'], ascending=False)
truncatedData = ordereddata.iloc[20:]

In [ ]:
free = truncatedData[truncatedData.price == 0]
paid10 = truncatedData[(truncatedData.price < 1001) & (truncatedData.price > 0)] #games between 0.01-10 dollars
paid20 = truncatedData[(truncatedData.price > 1000) & (truncatedData.price < 2001)]
paid30 = truncatedData[(truncatedData.price > 2000) & (truncatedData.price < 3001)]
paid40 = truncatedData[(truncatedData.price > 3000) & (truncatedData.price < 4001)]
paid50 = truncatedData[(truncatedData.price > 4000) & (truncatedData.price < 5001)]
paid60 = truncatedData[(truncatedData.price > 5000) & (truncatedData.price < 6001)]
paidOver60 = truncatedData[truncatedData.price > 6000]

In [ ]:
#owners of price points without top 20
priceOfGame = [free, paid10, paid20, paid30, paid40, paid50, paid60, paidOver60]
priceNames = ['Free', 'Paid 10', 'Paid 20', 'Paid 30', 'Paid 40', 'Paid 50', 'Paid 60', 'Paid Over 60']
dfPaid = createOwnersDF(priceOfGame, priceNames)
plotOwners(dfPaid)